## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout

files = os.listdir(path='data/')

## Types of recordings of Activities

In [2]:
devices = ["Respeck", "Thingy"]
student_numbers = ["s1923862", "s1915704", "s1971889"]
eating_static_activities = ["Sitting", "Standing"]
static_activities = ["Lying down right", "Lying down on left", "Lying down on stomach", "Lying down back"]
dynamic_activities = ["Shuffle walking", "Normal walking", "Running", "Ascending stairs", "Descending stairs", "Miscellaneous movements"]
sub_activities = ["Normal", "Talking", "Singing", "Coughing", "Laughing", "Hyperventilating"]
eating_sub_activity = "Eating"

f"{devices[0]}_{student_numbers[0]}_{static_activities[0]}_{sub_activities[0]}.csv"

'Respeck_s1923862_Lying down right_Normal.csv'

## Build consolidated DF of all recordings

In [3]:
def get_new_dataframe(student_number, activity, sub_activity, is_static):
    file = f"{devices[0]}_{student_number}_{activity}_{sub_activity}.csv"
    filename = "data/"+ file
    df = pd.read_csv(filename, index_col='Unnamed: 0')
    df['student_number'] = student_number
    df['activity'] = activity
    df['sub_activity'] = sub_activity
    df['is_static'] = is_static
    return df.drop('ind', axis=1)

dataset = pd.DataFrame()

for student_number in student_numbers:
    for activity in static_activities:
        for sub_activity in sub_activities:
            dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activity, 1)])

    for activity in eating_static_activities:
        for sub_activity in sub_activities:
            dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activity, 1)])
        dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, eating_sub_activity, 1)])

    for activity in dynamic_activities:
        dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activities[0], 0)])

dataset.head(10)

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,student_number,activity,sub_activity,is_static
0,1695993500435,0.866455,0.200378,0.483093,0.171875,-0.171875,0.093750,s1923862,Lying down right,Normal,1
1,1695993500475,0.865967,0.199890,0.478699,-0.109375,-0.062500,-0.125000,s1923862,Lying down right,Normal,1
2,1695993500515,0.861816,0.194519,0.488220,-0.046875,-0.390625,-0.156250,s1923862,Lying down right,Normal,1
3,1695993500555,0.869629,0.199158,0.488464,-0.109375,0.187500,0.109375,s1923862,Lying down right,Normal,1
4,1695993500595,0.862061,0.198181,0.488464,-0.093750,0.515625,0.218750,s1923862,Lying down right,Normal,1
5,1695993500635,0.864014,0.198669,0.485291,-0.218750,0.562500,0.156250,s1923862,Lying down right,Normal,1
6,1695993500675,0.865234,0.199158,0.476990,-0.140625,0.453125,0.203125,s1923862,Lying down right,Normal,1
7,1695993500715,0.869629,0.190857,0.488708,-1.546875,0.812500,0.062500,s1923862,Lying down right,Normal,1
8,1695993500755,0.861572,0.193542,0.485046,-0.140625,0.953125,0.265625,s1923862,Lying down right,Normal,1
9,1695993500795,0.866455,0.199402,0.478210,-0.546875,0.984375,0.343750,s1923862,Lying down right,Normal,1


## Sliding window to generate overlapping intervals of recordings
Interval size is temporarily set to 2 secs (ie, 50 recording points)

In [4]:
random_seed = 42   
n_time_steps = 50 
n_features = 3
step = 10
n_classes = 6 
n_epochs = 60       
batch_size = 1024   
learning_rate = 0.0025
l2_loss = 0.0015
segments = []
labels = []

x = 760
while x<dataset.shape[0]:
    for i in range(x-760,  x-n_time_steps, step):  
        accel_xs = dataset['accel_x'].values[i: i + n_time_steps]
        accel_ys = dataset['accel_y'].values[i: i + n_time_steps]
        accel_zs = dataset['accel_z'].values[i: i + n_time_steps]

        label = dataset['activity'].values[i]

        segments.append([accel_xs, accel_ys, accel_zs])

        labels.append(label)
    
    x+=760

#reshape the segments which is (list of arrays) to a list
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)


labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size = 0.2, random_state = random_seed)


## Build Multi-layer model
Modify hyperparameters

In [5]:
model = Sequential()
# RNN layer
model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
# # Dropout layer
model.add(Dropout(0.5))
# Dense layer with ReLu
model.add(Dense(units = 64, activation='relu'))
# Softmax layer
model.add(Dense(y_train.shape[1], activation = 'softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
Total params: 76620 (299.30 KB)
Trainable params: 76620 (299.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train model

In [6]:
model.fit(X_train, y_train, epochs = 100, validation_split = 0.20, batch_size = batch_size, verbose = 1)
model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)

Epoch 1/100
6/6 [==============================] - 2s 199ms/step - loss: 2.3940 - accuracy: 0.2520 - val_loss: 2.2218 - val_accuracy: 0.4079
Epoch 2/100
6/6 [==============================] - 1s 171ms/step - loss: 2.0844 - accuracy: 0.3663 - val_loss: 1.7552 - val_accuracy: 0.4126
Epoch 3/100
6/6 [==============================] - 1s 171ms/step - loss: 1.6590 - accuracy: 0.4141 - val_loss: 1.3819 - val_accuracy: 0.5343
Epoch 4/100
6/6 [==============================] - 1s 177ms/step - loss: 1.4084 - accuracy: 0.5102 - val_loss: 1.1880 - val_accuracy: 0.6573
Epoch 5/100
6/6 [==============================] - 1s 177ms/step - loss: 1.2290 - accuracy: 0.6240 - val_loss: 1.0450 - val_accuracy: 0.6989
Epoch 6/100
6/6 [==============================] - 1s 174ms/step - loss: 1.0928 - accuracy: 0.6641 - val_loss: 0.9389 - val_accuracy: 0.6714
Epoch 7/100
6/6 [==============================] - 1s 177ms/step - loss: 0.9956 - accuracy: 0.6700 - val_loss: 0.8930 - val_accuracy: 0.7144
Epoch 8/100
6

Epoch 59/100
6/6 [==============================] - 1s 193ms/step - loss: 0.4540 - accuracy: 0.8083 - val_loss: 0.3976 - val_accuracy: 0.8246
Epoch 60/100
6/6 [==============================] - 1s 179ms/step - loss: 0.4394 - accuracy: 0.8132 - val_loss: 0.3994 - val_accuracy: 0.8246
Epoch 61/100
6/6 [==============================] - 1s 182ms/step - loss: 0.4400 - accuracy: 0.8162 - val_loss: 0.3996 - val_accuracy: 0.8360
Epoch 62/100
6/6 [==============================] - 1s 186ms/step - loss: 0.4290 - accuracy: 0.8251 - val_loss: 0.3827 - val_accuracy: 0.8407
Epoch 63/100
6/6 [==============================] - 1s 188ms/step - loss: 0.4217 - accuracy: 0.8278 - val_loss: 0.3776 - val_accuracy: 0.8441
Epoch 64/100
6/6 [==============================] - 1s 185ms/step - loss: 0.4171 - accuracy: 0.8306 - val_loss: 0.3737 - val_accuracy: 0.8441
Epoch 65/100
6/6 [==============================] - 1s 178ms/step - loss: 0.4120 - accuracy: 0.8350 - val_loss: 0.3619 - val_accuracy: 0.8468
Epoch 

[0.3020567297935486, 0.8790972828865051]

The following cell is how we expect to be able to use the model to predict the activity

In [7]:
# How to predict activity
activities = ['Ascending stairs',
 'Descending stairs',
 'Lying down back',
 'Lying down on left',
 'Lying down on stomach',
 'Lying down right',
 'Miscellaneous movements',
 'Normal walking',
 'Running',
 'Shuffle walking',
 'Sitting',
 'Standing']

y_prd = model.predict(X_test)
activities[np.argmax(y_prd[0])]

59/59 [==============================] - 1s 6ms/step


'Standing'

In [8]:
confusion_matrix(list(map(np.argmax, y_test)), list(map(np.argmax, y_prd)))

array([[ 44,   0,   0,   0,   0,   0,   0,   0,   1,   4,   3,   0],
       [  0,  22,   0,   0,   0,   0,   0,   0,   0,   0,  23,  12],
       [  0,   0, 249,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 248,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 257,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 244,   0,   0,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,  17,   1,   2,   3,   4,   1],
       [  0,   0,   0,   0,   0,   0,   0,  34,   0,   1,   6,   0],
       [  0,   0,   0,   0,   0,   0,   1,   2,  49,   0,   0,   0],
       [  4,   7,   0,   0,   0,   0,   0,  15,   0,  12,   3,   2],
       [  4,   0,   0,   0,   0,   0,   0,  27,   0,   3, 203,  68],
       [  0,   2,  19,   0,   0,   0,   0,   0,   0,   2,   2, 257]])

## Export trained model to .tflite file

In [10]:
from tensorflow import lite
import tensorflow as tf
converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('lstm.tflite', 'wb').write(tfmodel)


INFO:tensorflow:Assets written to: /var/folders/40/nrlgy5ds3vjdypq153k6jqc00000gn/T/tmpd80bxals/assets


INFO:tensorflow:Assets written to: /var/folders/40/nrlgy5ds3vjdypq153k6jqc00000gn/T/tmpd80bxals/assets
2023-10-13 23:03:32.646617: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-13 23:03:32.646642: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-13 23:03:32.646907: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/40/nrlgy5ds3vjdypq153k6jqc00000gn/T/tmpd80bxals
2023-10-13 23:03:32.650408: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-13 23:03:32.650414: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/40/nrlgy5ds3vjdypq153k6jqc00000gn/T/tmpd80bxals
2023-10-13 23:03:32.659709: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-13 23:03:32.662976: I tensorflow/cc/saved_model/load

92752